## Import libraries and set random seed

In [1]:
from gurobipy import Model, GRB, tuplelist, quicksum
from gurobipy import read as gurobi_read_model
import json
import random
import math

random.seed(3)

## Required Functions

### PossibleStarts(object)

In [2]:
class PossibleStart(object):
    def __init__(self, resource, slice_starts, internal_start):
        self.resource = resource
        self.first_slice_start = slice_starts[0]
        self.all_slice_starts = slice_starts
        self.internal_start = internal_start

    def __lt__(self, other):
        return self.first_slice_start < other.first_slice_start

    def __eq__(self, other):
        return self.first_slice_start == self.first_slice_start

    def __gt__(self, other):
        return self.first_slice_start > self.first_slice_start

### InputParams(object)

In [3]:
class InputParams(object):
    def __init__(self, input_params_dict=None, requests = None):
        self.start_time = None
        self.slice_size = None
        self.horizon = None
        self.resources = None
        self.requests = None
        
        if input_params_dict != None:
            self.load_from_file(input_params_dict)

        if requests != None:
            self.load_requests(requests)
            
    def load_input_params(self, input_params_dict):
        self.start_time = input_params_dict.start_time
        self.slice_size = input_params_dict.slice_size
        self.horizon = input_params_dict.horizon
        self.resources = input_params_dict.resources

    def load_requests(self, requests):
        self.requests = requests

### get_slices(...)

In [4]:
def get_slices(intervals, resource, duration, slice_length):
    slices = []
    internal_starts = []
    for t in intervals: 
        start = int(math.floor(float(t["start"])/float(slice_length))*slice_length) # Start of the time_slice that this starts in
        internal_start = t["start"]                                                 # Actual start of this observation window
        end_time = internal_start + duration                                        # End of this observation window

        while (t["end"] - start) >= duration:
            # Generate a range of slices that will be occupied for this start and duration
            tmp = list(range(start, internal_start+duration, slice_length))
            slices.append(tmp)
            internal_starts.append(internal_start)
            start += slice_length                                # Moving on to the next potential start, so move the Start up to the next slice
            internal_start = start                               # As only the first potential start in a window will be internal, make the 
                                                                 # next Internal Start an external start
    # return slices, internal_starts
    ps_list = []
    idx = 0
    for w in slices:
        ps_list.append(PossibleStart(resource, w, internal_starts[idx]))
        idx += 1

    return ps_list

#### test

In [5]:
# TEST:
temp_slice_size = 300
intervals = [{"start": 0, "end": 30}, {"start": 60, "end": 90}]
possible_starts = get_slices(intervals, 13, 3, temp_slice_size)
print(possible_starts[1])

### time_segments(...)

In [6]:
def time_segments(start, end, num_min=1, num_max=5):
    segments = random.randint(num_min, num_max)
    boundaries = sorted([random.randint(start, end) for x in range(2*segments)])
    windows = [{"start": boundaries[i], "end": boundaries[i+1]} for i in range(0, len(boundaries), 2)]
    return windows

#### test

In [7]:
ts1 = time_segments(0, 10, 2, 2)
ts2 = time_segments(0, 10, 2, 2)
print(ts1)
print(ts2)

[{'start': 2, 'end': 5}, {'start': 8, 'end': 9}]
[{'start': 1, 'end': 9}, {'start': 9, 'end': 10}]


### overlap_time_segments(...)

In [8]:
def overlap_time_segments(seg1, seg2):
    overlap_segments = []
    i = 0
    j = 0

    while ((i < len(seg1)) and (j < len(seg2))):
        s1 = seg1[i]
        s2 = seg2[j]

        if s1["end"] < s2["start"]: # Current s1 is behind Current s2, move to next s1
            i += 1
            continue

        if s2["end"] < s1["start"]: # Current s2 is behind Current s1, move to next s2
            j += 1
            continue

        window_start = max([s1["start"], s2["start"]])
        
        if s1["end"] < s2["end"]: # s1 ends first, move to next s1
            window_end = s1["end"]
            i += 1
            
        elif s1["end"] > s2["end"]: # s2 ends first, move to next s2
            window_end= s2["end"]
            j += 1

        else: # Both segments end at the same time, move to next of both
            window_end = s1["end"]
            i += 1
            j+= 1

        if window_start != window_end:
            overlap_segments.append({"start": window_start, "end": window_end})
            
    return overlap_segments

#### test

In [9]:
ts1 = [{"start": 2, "end": 4}, {"start": 5, "end": 6}, {"start": 8, "end": 9}]
ts2 = [{"start": 0, "end": 3}, {"start": 4, "end": 8}]

overlap_time_segments(ts1, ts2)

[{'start': 2, 'end': 3}, {'start': 5, 'end': 6}]

### build_data_structures(...)

Build the Yik and the aikt for generating the Optimizer Constraints from.

Requests must be a list of requests, which should have:
* free_windows_dict
* duration
* resID
* priority

In [10]:
def build_data_structures(requests, slice_length):
    yik = []
    aikt = {}

    for i in range(len(requests)):
        r = requests[i]
        yik_entries = []
        possible_starts = []

        print(r)
        for resource in r["free_windows_dict"].keys():
            possible_starts.extend(get_slices(r["free_windows_dict"][resource], resource, r["duration"], slice_length))
        possible_starts.sort()
    
        w_idx = 0
        for ps in possible_starts:
            yik_idx = len(yik)
            yik_entries.append(yik_idx)
            scheduled = 0 # Option to set to 1 for a Warm Start would go here
            yik.append([r["resID"], w_idx, r["priority"], ps.resource, scheduled, ps])
            w_idx += 1
    
            for s in ps.all_slice_starts:
                slice_hash = f"resource_{ps.resource}_start_{repr(s)}_length_{repr(slice_length)}"
                if slice_hash not in aikt:
                    aikt[slice_hash] = []
                aikt[slice_hash].append(yik_idx)
                
    return (yik, aikt)

#### test

In [11]:
test_requests = [
    {"free_windows_dict": {"t1": [{"start": 0, "end": 23}, {"start": 27, "end": 88}]},
     "duration": 10,
     "resID": 3,
     "priority": 5},
    {"free_windows_dict": {"t1": [{"start": 24, "end": 63}]},
     "duration": 13,
     "resID": 1,
     "priority": 4}
]

build_data_structures(test_requests, 5)

{'free_windows_dict': {'t1': [{'start': 0, 'end': 23}, {'start': 27, 'end': 88}]}, 'duration': 10, 'resID': 3, 'priority': 5}
{'free_windows_dict': {'t1': [{'start': 24, 'end': 63}]}, 'duration': 13, 'resID': 1, 'priority': 4}


([[3, 0, 5, 't1', 0, <__main__.PossibleStart at 0x1982c153390>],
  [3, 1, 5, 't1', 0, <__main__.PossibleStart at 0x1982c010710>],
  [3, 2, 5, 't1', 0, <__main__.PossibleStart at 0x1982c1504d0>],
  [3, 3, 5, 't1', 0, <__main__.PossibleStart at 0x1982c151cd0>],
  [3, 4, 5, 't1', 0, <__main__.PossibleStart at 0x1982c152050>],
  [3, 5, 5, 't1', 0, <__main__.PossibleStart at 0x1982c1009d0>],
  [3, 6, 5, 't1', 0, <__main__.PossibleStart at 0x1982c102fd0>],
  [3, 7, 5, 't1', 0, <__main__.PossibleStart at 0x1982c100250>],
  [3, 8, 5, 't1', 0, <__main__.PossibleStart at 0x1982c101c90>],
  [3, 9, 5, 't1', 0, <__main__.PossibleStart at 0x1982c1000d0>],
  [3, 10, 5, 't1', 0, <__main__.PossibleStart at 0x1982c1031d0>],
  [3, 11, 5, 't1', 0, <__main__.PossibleStart at 0x1982c101690>],
  [3, 12, 5, 't1', 0, <__main__.PossibleStart at 0x1982c101950>],
  [3, 13, 5, 't1', 0, <__main__.PossibleStart at 0x1982c0cffd0>],
  [1, 0, 4, 't1', 0, <__main__.PossibleStart at 0x1982c0cf1d0>],
  [1, 1, 4, 't1', 0, 

### build_model(...)

In [12]:
def build_model(yik, aikt, filename="test_model.mps"):
    m = Model("Test Schedule")

    requestLocations = tuplelist()
    scheduled_vars = []

    for r in yik:
        var = m.addVar(vtype=GRB.BINARY, name=str(r[0]))
        scheduled_vars.append(var)
        requestLocations.append((r[0], r[1], r[2], r[3], var)) # resID, start_w_idx, priority, resource, isScheduled?
    m.update()

    # Constraint 4: Each request only scheduled once
    for rid in requests:
        match = requestLocations.select(rid, '*', '*', '*', '*', '*')
        nscheduled = quicksum([isScheduled for reqid, wnum, priority, resource, isScheduled in match])
        m.addConstr(nscheduled <= 1, f'one_per_reqid_constraint_{rid}')
    m.update()

    # Constraint 3: Each timeslice should only have one request in it
    for s in aikt.keys():
        match = tuplelist()
        for timeslice in aikt[s]:
            match.append(requestLocations[timeslice])
        nscheduled = quicksum(isScheduled for reqid, winidx, priority, resource, isScheduled in match)
        m.addConstr(nscheduled <= 1, f"one_per_slice_constrain_{s}")

    objective = quicksum([isScheduled * (priority + 0.1/(winidx+1.0)) for req, winidx, priority, resource, isScheduled in requestLocations])

    m.setObjective(objective)
    m.modelSense = GRB.MAXIMIZE

    # Implement a timelimit? (Do I actually want to do this?)
    timelimit = 0 # NOTE: Hardcode out for now.
    if timelimit > 0:
        m.params.timeLimit = timelimit

    # Set the tolerance of the solution
    m.params.MIPGap = 0.01

    # Set the Method of solving the root relaxation of the MIPs model to concurrent
    m.params.Method = 3

    m.update()

    m.write(filename)

    print(f"\n\nModel built and written to file: {filename}")

### load_model()

In [13]:
def load_model(model_name="test_model.mps"):
    model = gurobi_read_model(model_name)
    return model

### interpret_solution(...)

In [14]:
def interpret_solution(solution, yik, aikt, requests):
    # Get list of all requests that are scheduled
    # Get details of scheduled requests
    # Sort by resource and start time
    # Print Details

    enum_aikt = list(enumerate(aikt))
    scheduled = []
    for i in range(len(solution)):
        if solution[i] == 1:
            scheduled.append(yik[i]) # ID, start_w_idx, priority, resource, isScheduled(dud), possible_start

    scheduled.sort(key=lambda x: [x[3], x[1]]) # Sort by Resource, then by Starting Window
    for i in range(len(scheduled)):
        
        s = scheduled[i]
        rid = s[0]

        print(i)
        print(s)
        print(rid)
        
        r = requests[rid]
        print(f"Request {rid}:")
        print(f"Resource = {s[3]}")
        start_time = s[5].internal_start
        duration = r["duration"]
        end_time = start_time + duration
        print(f"Start Window ID = {enum_aikt[s[1]-1]}")
        print(f"Start / End (Duration) = {start_time} / {end_time} ({duration})")
        print(f"Duration = {r['duration']}")
        
        print()

In [16]:
# scheduled = []
# for i in range(len(solution)):
#     if solution[i] == 1:
#         scheduled.append(yik[i])

In [17]:
scheduled

[]

In [18]:
for s in scheduled:
    print(s[5].internal_start)
    print(s[5].first_slice_start)
    print(s[5].all_slice_starts)

## Input Parameters

### Scheduler Parameters

In [19]:
input_params = InputParams()
input_params.start_time = 0
input_params.slice_size = 300
input_params.horizon = 24*60*60
print(f"Slice_size: {input_params.slice_size}s")
print(f"Scheduling Horizon: {input_params.horizon}s")

Slice_size: 300s
Scheduling Horizon: 86400s


### Request-Generation Parameters

In [20]:
num_requests = 20
request_min_length = 30
request_max_length = 60*60*3
priority_min = 1
priority_max = 10

### Resource Availability

In [22]:
resources = ["t1"]
resource_availability = {res: time_segments(input_params.start_time, input_params.horizon, 1, 5) for res in resources}
resource_availability

{'t1': [{'start': 33994, 'end': 61503}]}

### Generate Observation Requests (Randomly)

In [23]:
# Generate random requests
requests = {}

for i in range(num_requests):
    requests[i] = {
        "windows": {r: time_segments(input_params.start_time, input_params.start_time+input_params.horizon, 1, 8) for r in resources},
        "duration": random.randint(request_min_length, request_max_length),
        "priority": random.randint(priority_min, priority_max),
        "resID": i
    }

for r in requests.items():
    print(r)

(0, {'windows': {'t1': [{'start': 19741, 'end': 25132}, {'start': 52053, 'end': 61638}, {'start': 62436, 'end': 70906}, {'start': 72041, 'end': 83763}]}, 'duration': 3829, 'priority': 3, 'resID': 0})
(1, {'windows': {'t1': [{'start': 1985, 'end': 4064}, {'start': 5608, 'end': 8392}, {'start': 20892, 'end': 35314}, {'start': 39487, 'end': 50804}, {'start': 51768, 'end': 55959}, {'start': 61964, 'end': 75616}, {'start': 77476, 'end': 77955}]}, 'duration': 7314, 'priority': 3, 'resID': 1})
(2, {'windows': {'t1': [{'start': 4703, 'end': 12773}, {'start': 17821, 'end': 28440}, {'start': 33814, 'end': 39456}, {'start': 50576, 'end': 55200}, {'start': 57168, 'end': 64865}, {'start': 66485, 'end': 82136}]}, 'duration': 9434, 'priority': 6, 'resID': 2})
(3, {'windows': {'t1': [{'start': 3756, 'end': 13641}, {'start': 21377, 'end': 27672}, {'start': 30459, 'end': 36658}, {'start': 42780, 'end': 44140}, {'start': 71010, 'end': 74594}, {'start': 74967, 'end': 76579}, {'start': 79405, 'end': 85919}

## Model

### Overlap Request Windows and Resource Windows

In [24]:
for i, r in requests.items():
    requests[i]["free_windows_dict"] = {}
    for res in requests[i]["windows"]:
        requests[i]["free_windows_dict"][res] = overlap_time_segments(resource_availability[res], requests[i]["windows"][res])
requests

{0: {'windows': {'t1': [{'start': 19741, 'end': 25132},
    {'start': 52053, 'end': 61638},
    {'start': 62436, 'end': 70906},
    {'start': 72041, 'end': 83763}]},
  'duration': 3829,
  'priority': 3,
  'resID': 0,
  'free_windows_dict': {'t1': [{'start': 52053, 'end': 61503}]}},
 1: {'windows': {'t1': [{'start': 1985, 'end': 4064},
    {'start': 5608, 'end': 8392},
    {'start': 20892, 'end': 35314},
    {'start': 39487, 'end': 50804},
    {'start': 51768, 'end': 55959},
    {'start': 61964, 'end': 75616},
    {'start': 77476, 'end': 77955}]},
  'duration': 7314,
  'priority': 3,
  'resID': 1,
  'free_windows_dict': {'t1': [{'start': 33994, 'end': 35314},
    {'start': 39487, 'end': 50804},
    {'start': 51768, 'end': 55959}]}},
 2: {'windows': {'t1': [{'start': 4703, 'end': 12773},
    {'start': 17821, 'end': 28440},
    {'start': 33814, 'end': 39456},
    {'start': 50576, 'end': 55200},
    {'start': 57168, 'end': 64865},
    {'start': 66485, 'end': 82136}]},
  'duration': 9434,
 

### Build Model

In [25]:
yik, aikt = build_data_structures(requests, 300)
build_model(yik, aikt)

{'windows': {'t1': [{'start': 19741, 'end': 25132}, {'start': 52053, 'end': 61638}, {'start': 62436, 'end': 70906}, {'start': 72041, 'end': 83763}]}, 'duration': 3829, 'priority': 3, 'resID': 0, 'free_windows_dict': {'t1': [{'start': 52053, 'end': 61503}]}}
{'windows': {'t1': [{'start': 1985, 'end': 4064}, {'start': 5608, 'end': 8392}, {'start': 20892, 'end': 35314}, {'start': 39487, 'end': 50804}, {'start': 51768, 'end': 55959}, {'start': 61964, 'end': 75616}, {'start': 77476, 'end': 77955}]}, 'duration': 7314, 'priority': 3, 'resID': 1, 'free_windows_dict': {'t1': [{'start': 33994, 'end': 35314}, {'start': 39487, 'end': 50804}, {'start': 51768, 'end': 55959}]}}
{'windows': {'t1': [{'start': 4703, 'end': 12773}, {'start': 17821, 'end': 28440}, {'start': 33814, 'end': 39456}, {'start': 50576, 'end': 55200}, {'start': 57168, 'end': 64865}, {'start': 66485, 'end': 82136}]}, 'duration': 9434, 'priority': 6, 'resID': 2, 'free_windows_dict': {'t1': [{'start': 33994, 'end': 39456}, {'start':

### Solve Model - Gurobi

In [26]:
model = load_model()
model.optimize()

Read MPS format model from file test_model.mps
Reading time = 0.03 seconds
Test: 112 rows, 363 columns, 6366 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 112 rows, 363 columns and 6366 nonzeros
Model fingerprint: 0x4eb34b59
Variable types: 0 continuous, 363 integer (363 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 23.3159415
Presolve removed 59 rows and 163 columns
Presolve time: 0.08s
Presolved: 53 rows, 200 columns, 2489 nonzeros
Variable types: 0 continuous, 200 integer (200 binary)
Found heuristic solution: objective 34.2438095

Root relaxation: objective 4.271720e+01, 62 iterations, 0.00 seconds (0.00 work units)

    Node

In [27]:
if model.Status == GRB.OPTIMAL:
    solution = model.getAttr("X")
    for i in range(len(solution)):
        if solution[i] == 1:
            print(i, yik[i])

72 [6, 0, 7, 't1', 0, <__main__.PossibleStart object at 0x000001982C747A50>]
98 [7, 14, 9, 't1', 0, <__main__.PossibleStart object at 0x000001982C731150>]


In [28]:
print("Total Priority Score: ", sum(solution))

Total Priority Score:  6.000000499359214


## Interpret Results

In [29]:
interpret_solution(solution, yik, aikt, requests)

0
[6, 0, 7, 't1', 0, <__main__.PossibleStart object at 0x000001982C747A50>]
6
Request 6:
Resource = t1
Start Window ID = (91, 'resource_t1_start_51600_length_300')
Start / End (Duration) = 33994 / 37257 (3263)
Duration = 3263

1
[7, 14, 9, 't1', 0, <__main__.PossibleStart object at 0x000001982C731150>]
7
Request 7:
Resource = t1
Start Window ID = (13, 'resource_t1_start_55800_length_300')
Start / End (Duration) = 53700 / 61273 (7573)
Duration = 7573



## Create Sample Input File

### Create Input Variables

In [30]:
sample_input_variables = {
    "start_time": 0,
    "slice_size": 300,
    "horizon": 28800,    # 8 hours
    "resources": {"t1": [{"start": 0, "end": 10000}, {"start": 12500, "end": 28800}]}
}

### Create Requests

In [31]:
sample_requests = {}
#0
sample_requests[0] = {
    "windows": {
        "t1": [
            {'start': 0, "end": 4500},
            {'start': 8100, "end": 12600},
            {'start': 16200, "end": 20700},
            {'start': 24300, "end": 28800}
        ]
    },
    "duration": 2000,
    "priority": 3,
    "resID": 0
}

In [32]:
#1
sample_requests[1] = {
    "windows": {
        "t1": [
            {'start': 2000, "end": 8000},
            {'start': 13000, "end": 19000}
        ]
    },
    "duration": 4000,
    "priority": 3,
    "resID": 1
}

In [33]:
#2
sample_requests[2] = {
    "windows": {
        "t1": [
            {'start': 0, "end": 1800},
            {'start': 3600, "end": 5400},
            {'start': 7200, "end": 9000},
            {'start': 10800, "end": 12600},
            {'start': 14400, "end": 16200}
        ]
    },
    "duration": 1800,
    "priority": 10,
    "resID": 2
}

In [34]:
#3
sample_requests[3] = {
    "windows": {
        "t1": [
            {'start': 50, "end": 9050},
        ]
    },
    "duration": 6000,
    "priority": 3,
    "resID": 3
}

In [35]:
#4
sample_requests[4] = {
    "windows": {
        "t1": [
            {'start': 2000, "end": 8000},
            {'start': 14500, "end": 26800}
        ]
    },
    "duration": 1000,
    "priority": 3,
    "resID": 4
}

In [36]:
#5
sample_requests[5] = {
    "windows": {
        "t1": [
            {'start': 0, "end": 1000},
            {'start': 9500, "end": 10500},
            {'start': 14500, "end": 15500},
            {'start': 25500, "end": 26500}
        ]
    },
    "duration": 400,
    "priority": 3,
    "resID": 5
}

### Combine and Save

In [37]:
sample_input = {
    "input_parameters": sample_input_variables,
    "requests": sample_requests
}
sample_input

{'input_parameters': {'start_time': 0,
  'slice_size': 300,
  'horizon': 28800,
  'resources': {'t1': [{'start': 0, 'end': 10000},
    {'start': 12500, 'end': 28800}]}},
 'requests': {0: {'windows': {'t1': [{'start': 0, 'end': 4500},
     {'start': 8100, 'end': 12600},
     {'start': 16200, 'end': 20700},
     {'start': 24300, 'end': 28800}]},
   'duration': 2000,
   'priority': 3,
   'resID': 0},
  1: {'windows': {'t1': [{'start': 2000, 'end': 8000},
     {'start': 13000, 'end': 19000}]},
   'duration': 4000,
   'priority': 3,
   'resID': 1},
  2: {'windows': {'t1': [{'start': 0, 'end': 1800},
     {'start': 3600, 'end': 5400},
     {'start': 7200, 'end': 9000},
     {'start': 10800, 'end': 12600},
     {'start': 14400, 'end': 16200}]},
   'duration': 1800,
   'priority': 10,
   'resID': 2},
  3: {'windows': {'t1': [{'start': 50, 'end': 9050}]},
   'duration': 6000,
   'priority': 3,
   'resID': 3},
  4: {'windows': {'t1': [{'start': 2000, 'end': 8000},
     {'start': 14500, 'end': 26

In [38]:
with open("sample_input.json", "w") as f:
    json.dump(sample_input, f)

## Convert Input File to Parameters